In [1]:
import os
import torch
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
from itertools import product
import json
from collections import defaultdict
import random

from torch import nn
from torch.autograd import Variable
from torch.functional import F
from torchvision import models
import torchvision
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np

# %matplotlib inline
# %reload_ext autoreload
# %autoreload 2

# from fastai import transforms, model, dataset, conv_learner

from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

torch.set_printoptions(precision=3)

from SSD_model import get_SSD_model
from VOC_data import VOC_dataset
from draw_img_utils import *
from SSDloss import *

In [60]:
pred = torch.randn(21) * 200
pred = pred.unsqueeze(0)

res = torch.LongTensor([16])

In [61]:
c_loss = nn.CrossEntropyLoss(reduce=False)
# c_loss = nn.CrossEntropyLoss()
c_loss(pred, res)

D:\py3\anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


tensor([717.239])

In [25]:
s_pred = F.softmax(pred)

D:\py3\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [26]:
s_pred

tensor([[0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
         0.804, 0.000, 0.000, 0.196, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
         0.000]])

In [27]:
pred

tensor([[ -22.755,  -16.453,    5.476,  -40.242,  -58.208,   11.887,   26.608,
         -132.171,   23.920,   30.792,   95.328,   54.485,  -25.477,   93.916,
          -48.011,  -48.178,   16.141,    8.699,  -31.077,   51.406,   10.143]])

In [30]:
sum_pred = torch.sum(torch.exp(pred - torch.max(pred)))

In [31]:
sum_pred

tensor(1.244)

In [32]:
sofrmax_res = torch.exp(pred - torch.max(pred)) / sum_pred

In [33]:
sofrmax_res

tensor([[0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
         0.804, 0.000, 0.000, 0.196, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000,
         0.000]])

In [34]:
torch.max(pred)

tensor(95.328)

In [36]:
torch.exp(torch.tensor(-132.171 - 95.328))

tensor(0.)

In [41]:
torch.log(sofrmax_res)

tensor([[   -inf,    -inf, -90.070,    -inf,    -inf, -83.659, -68.938,    -inf,
         -71.626, -64.754,  -0.218, -41.061,    -inf,  -1.630,    -inf,    -inf,
         -79.405, -86.847,    -inf, -44.140, -85.403]])

In [67]:
x = torch.randn(2, 4)
w = nn.Parameter(torch.randn(4, 2))
b = nn.Parameter(torch.randn(2, 1))
y = torch.randn(2, 2)

In [ ]:
model = nn.Para

In [68]:
pred = torch.matmul(x, w) + b

In [69]:
pred

tensor([[ 0.233, -0.430],
        [ 1.184,  0.563]], grad_fn=<ThAddBackward>)

In [81]:
pred1 = torch.matmul(x[0], w) + b[0]
pred1

tensor([ 0.233, -0.430], grad_fn=<ThAddBackward>)

In [82]:
pred2 = torch.matmul(x[1], w) + b[1]
pred2

tensor([1.184, 0.563], grad_fn=<ThAddBackward>)

In [86]:
optimizer = torch.optim.Adam(nn.ParameterList((w, b)))
criterion = nn.SmoothL1Loss()

In [99]:
loss = criterion(pred, y)

In [100]:
loss

tensor(0.217, grad_fn=<SmoothL1LossBackward>)

In [89]:
loss.backward(retain_graph=True)

In [90]:
w.grad

tensor([[ 0.139,  0.082],
        [ 0.258,  0.242],
        [-0.265, -0.173],
        [-0.047,  0.030]])

In [92]:
w.grad = torch.zeros(4, 2)
b.grad = torch.zeros(2, 1)

In [93]:
w.grad

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [94]:
b.grad

tensor([[0.],
        [0.]])

In [95]:
l1 = criterion(pred1, y[0])

In [96]:
l1

tensor(0.054, grad_fn=<SmoothL1LossBackward>)

In [97]:
l2 = criterion(pred2, y[1])

In [98]:
l2

tensor(0.381, grad_fn=<SmoothL1LossBackward>)

In [102]:
l = (l1 + l2) / 2

In [103]:
l

tensor(0.217, grad_fn=<DivBackward0>)

In [104]:
l.backward(retain_graph=True)

In [105]:
w.grad

tensor([[ 0.139,  0.082],
        [ 0.258,  0.242],
        [-0.265, -0.173],
        [-0.047,  0.030]])

In [111]:
import os
import torch
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
from itertools import product
import json
from collections import defaultdict
import random

from torch import nn
from torch.autograd import Variable
from torch.functional import F
from torchvision import models
import torchvision
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np

# %matplotlib inline
# %reload_ext autoreload
# %autoreload 2

# from fastai import transforms, model, dataset, conv_learner

from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

torch.set_printoptions(precision=3)

from SSD_model import get_SSD_model
from VOC_data import VOC_dataset
from draw_img_utils import *
from SSDloss import *

torch.set_printoptions(precision=3)

In [107]:
PATH = 'C:\\datasets\\pascal\\'
anno_path = f'{PATH}PASCAL_VOC\\pascal_train2007.json'
train_dataset = VOC_dataset(PATH, anno_path)

img, bbox, label = train_dataset[7]
img = img.unsqueeze(0)

prior_box = get_prior_box()
iou = get_iou(bbox, prior_box)

pos_mask, cls_target, bbox_target = get_target(iou, prior_box, img, bbox, label)

model = get_SSD_model(1)
cls_pred, loc_pred = model(img)
cls_pred, loc_pred = cls_pred.squeeze(0), loc_pred.squeeze(0)

In [117]:
prior_box = get_prior_box()
iou = get_iou(bbox, prior_box)
pos_mask, cls_target, bbox_target = get_target(iou, prior_box, img, bbox, label)

ratio = 3
num_pos = torch.sum(pos_mask)
num_neg = ratio * num_pos

# if num_pos == 0:
#     return torch.tensor(0)

# loss of class including hard negative mining
loc_criterion, cls_criterion = nn.SmoothL1Loss(), nn.CrossEntropyLoss(reduce=False)


loss_cls_pos = cls_criterion(cls_pred[pos_mask], cls_target.long()[pos_mask])
loss_cls_pos = torch.sum(loss_cls_pos)

conf_loss_neg = cls_criterion(cls_pred[1 - pos_mask], cls_target.long()[1 - pos_mask])
val, arg = torch.sort(conf_loss_neg, descending=True)
loss_cls_neg = torch.sum(conf_loss_neg[arg[:num_neg]])

loss_cls = loss_cls_neg + loss_cls_pos 

# loss of location
loss_loc = loc_criterion(loc_pred[pos_mask], bbox_target[pos_mask])
loss = loss_loc + loss_cls / float(num_pos)

print(loss_cls.data / float(num_pos), loss_loc)

D:\py3\anaconda3\lib\site-packages\torch\nn\functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


tensor(320.754) tensor(3.418, grad_fn=<SmoothL1LossBackward>)


In [114]:
loc_pred[pos_mask]

tensor([[ 4.226e+00,  1.618e+01,  3.050e+01,  1.663e+01],
        [ 5.867e-01, -6.607e-01, -1.243e-01, -2.324e+00],
        [ 1.057e+00, -9.935e-01, -3.803e-01, -1.221e+00],
        [ 1.064e+00,  1.511e-01, -5.674e-01,  4.986e-03],
        [ 9.808e-01, -1.007e+00, -3.672e-01, -1.416e+00],
        [ 1.729e+00,  1.612e+00, -3.766e-01, -5.434e-01],
        [-6.616e-01, -1.868e-01, -6.477e-01, -1.707e+00],
        [ 2.729e+00,  2.480e+00, -6.058e-01, -1.105e+00],
        [-1.043e+00,  1.473e+00,  2.648e+00, -5.009e+00],
        [-3.756e-01,  7.795e-01,  2.583e+00, -3.655e+00],
        [ 9.107e-01,  2.989e-01,  2.190e+00, -2.428e+00],
        [ 8.180e-01,  2.958e-01,  4.789e-01, -2.304e+00],
        [ 4.447e-01, -1.195e+00,  7.327e-01, -1.351e+00],
        [ 1.185e+00,  6.343e-03, -5.125e-01, -6.225e-01],
        [ 2.489e-01, -5.643e-01,  1.083e+00, -1.102e+00],
        [ 9.190e-01,  1.428e+00, -6.697e-01, -1.986e+00],
        [-1.236e+00,  5.023e-01,  7.296e-01, -1.508e+00],
        [ 8.50

In [115]:
bbox_target[pos_mask]

tensor([[ 4.179, 11.892, -3.810,  4.206],
        [-5.194,  4.828,  0.945,  0.574],
        [-4.362,  0.251,  0.945,  0.574],
        [-5.343,  0.205,  1.958, -0.440],
        [-4.362,  0.667,  0.945,  0.574],
        [-5.343,  0.885,  1.958, -0.440],
        [-4.362,  1.500,  0.945,  0.574],
        [-5.343,  1.224,  1.958, -0.440],
        [11.165,  4.796,  2.251, -0.353],
        [11.165,  5.628,  2.251, -0.353],
        [11.165,  6.044,  2.251, -0.353],
        [11.165,  6.877,  2.251, -0.353],
        [-3.530, -3.910,  0.945,  0.574],
        [-4.323, -2.852,  1.958, -0.440],
        [-3.530, -3.077,  0.945,  0.574],
        [-4.323, -2.513,  1.958, -0.440],
        [-3.530, -2.661,  0.945,  0.574],
        [-4.323, -1.833,  1.958, -0.440],
        [11.997,  1.051,  2.251, -0.353],
        [11.997,  1.467,  2.251, -0.353],
        [11.997,  2.300,  2.251, -0.353],
        [11.997,  2.716,  2.251, -0.353],
        [-3.530,  8.157,  0.945,  0.574],
        [-4.323,  6.660,  1.958, -

In [118]:
loc_criterion(loc_pred, bbox_target)

tensor(15.292, grad_fn=<SmoothL1LossBackward>)